In [20]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [31]:
import numpy as np

from stockMarket.core.contract import Contract
from stockMarket.api import populate_contracts

contract = Contract(ticker='AAPL')
contracts = populate_contracts([contract])
print(contracts[0].full_pricing_info)
print(contracts[0].get_price_by_date(years_back=1))

test
             datetime\tsymbol\topen\thigh\tlow\tclose\tvolume
1970-01-01  2004-04-07 15:30:00\t:AAPL\t0.49\t0.49\t0.48\t...
1970-01-01  2004-04-08 15:30:00\t:AAPL\t0.50\t0.50\t0.49\t...
1970-01-01  2004-04-12 15:30:00\t:AAPL\t0.49\t0.50\t0.49\t...
1970-01-01  2004-04-13 15:30:00\t:AAPL\t0.50\t0.50\t0.48\t...
1970-01-01  2004-04-14 15:30:00\t:AAPL\t0.48\t0.48\t0.47\t...
...                                                       ...
1970-01-01  2024-02-12 15:30:00\t:AAPL\t188.41\t188.67\t18...
1970-01-01  2024-02-13 15:30:00\t:AAPL\t185.77\t186.21\t18...
1970-01-01  2024-02-14 15:30:00\t:AAPL\t185.32\t185.53\t18...
1970-01-01  2024-02-15 15:30:00\t:AAPL\t183.55\t184.49\t18...
1970-01-01  2024-02-16 15:30:00\t:AAPL\t183.42\t184.85\t18...

[5000 rows x 1 columns]


ValueError: No axis named close for object type Series

### Retrieve Data

In [5]:
from stockMarket.core import get_tickers_from_index
from stockMarket.api import populate_contracts

tickers = get_tickers_from_index('sp500')
contracts = populate_contracts(tickers)

### Perform Screening

In [7]:
import numpy as np
import operator

from stockMarket.core import Ranking, RangeRankingObject
from stockMarket.core.ranking import ValueRankingConstraint

equity_ratio_ranking = RangeRankingObject("Equity Ratio", lambda x: x.equity_ratio, [10, 30], [0,1,2])
netto_margin_ranking = RangeRankingObject("Netto Margin", lambda x: x.netto_margin, [10, 20], [0,1,2])
return_on_assets_ranking = RangeRankingObject("Return on Assets", lambda x: x.return_on_assets, [5, 10], [0,1,2])
goodwill_ranking = RangeRankingObject("Goodwill", lambda x: x.goodwill_ratio, [0, 30], [0,1,0])
gearing_ranking = RangeRankingObject("Gearing", lambda x: x.gearing, [20, 60], [2,1,0])

trailing_pe_constraint = ValueRankingConstraint(lambda x: x.price_to_earnings, operator.gt, 0)
peg_trailing_3y_ranking = RangeRankingObject(
    "PEG Trailing 3Y",
    lambda x: x.peg_trailing_3y,
    [0.8, 1.2], [2,1,0],
    constraints=[trailing_pe_constraint]
)

trailing_pr_constraint = ValueRankingConstraint(lambda x: x.price_to_revenue, operator.gt, 0)
prg_trailing_3y_ranking = RangeRankingObject(
    "PRG Trailing 3Y",
    lambda x: x.prg_trailing_3y,
    [0.8, 1.2], [2,1,0],
    constraints=[trailing_pr_constraint]
)

trailing_pfc_constraint = ValueRankingConstraint(lambda x: x.price_to_free_cashflow, operator.gt, 0)
pfcg_trailing_3y_ranking = RangeRankingObject(
    "PFCG Trailing 3Y",
    lambda x: x.pfcg_trailing_3y,
    [0.8, 1.2], [2,1,0],
    constraints=[trailing_pfc_constraint]
)



ranking_list = [
    equity_ratio_ranking,
    netto_margin_ranking,
    return_on_assets_ranking,
    goodwill_ranking,
    gearing_ranking,
    peg_trailing_3y_ranking,
    prg_trailing_3y_ranking,
    pfcg_trailing_3y_ranking,
]



ranking = Ranking(contracts, ranking_list, years_back=0)
ranking.rank()
ranking.ranking.to_csv("ranking_contracts.csv", sep='\t', encoding='utf-8', float_format='%.2f')
ranking.ranking

,Name,Sector,Relative Score,Absolute Score,No Data/Constraints/Tot.,Equity Ratio,Equity Ratio Score,Netto Margin,Netto Margin Score,Return on Assets,...,Goodwill,Goodwill Score,Gearing,Gearing Score,PEG Trailing 3Y,PEG Trailing 3Y Score,PRG Trailing 3Y,PRG Trailing 3Y Score,PFCG Trailing 3Y,PFCG Trailing 3Y Score
ABNB,"Airbnb, Inc.",Consumer Cyclical,100.00%,15/15,0/0/8,34.667664,2,22.538397,2,11.803217,...,11.690647,1,-111.097122,2,-0.221964,2,0.659735,2,0.263200,2
CTRA,Coterra Energy Inc.,Energy,100.00%,15/15,0/0/8,62.865932,2,44.912164,2,20.169693,...,0.000000,1,17.103394,2,0.102322,2,0.065315,2,0.143798,2
PXD,Pioneer Natural Resources Company,Energy,100.00%,15/15,0/0/8,63.069390,2,32.172736,2,21.950196,...,1.078036,1,19.426822,2,0.075575,2,0.106016,2,0.183373,2
BLK,"BlackRock, Inc.",Financial Services,100.00%,2/2,7/0/8,NaN,0,30.807996,2,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
TROW,"T. Rowe Price Group, Inc.",Financial Services,100.00%,15/15,0/0/8,75.919198,2,24.010542,2,13.380227,...,29.897619,1,-12.387578,2,-1.828060,2,0.569829,2,0.567650,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PPG,"PPG Industries, Inc.",Basic Materials,0.00%,0/2,7/0/8,NaN,0,6.960430,0,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
PKG,Packaging Corporation of America,Consumer Cyclical,0.00%,0/2,7/0/8,NaN,0,9.807239,0,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
MS,Morgan Stanley,Financial Services,0.00%,0/2,7/0/8,NaN,0,9.446535,0,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
MRK,"Merck & Co., Inc.",Healthcare,0.00%,0/2,7/0/8,NaN,0,0.607170,0,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0


### Write Email

In [40]:
from stockMarket.utils import write_email

emails = ["benjaminlantschner@gmail.com", "97gamjak@gmail.com"]
subject = "Stock Market Ranking"
body = ""
attachment = "ranking_contracts.csv"
write_email(emails, subject, body, attachment)